#### Block Processing

In [10]:
import os 
from glob import glob
from geotile import mosaic

##### Merge

block merge [x]:
- gdem v and f 
- tdem v and f 


In [ ]:
tilenames_tls = [
 'N11E104',
 'N11E105',
 'N12E103',
 'N12E104',
 'N12E105',
 'N13E103',
 'N13E104',
 'N13E105']


grid_filesi = glob("/media/ljp238/12TBWolf/RSPROX/OUT_TILES/TILES12/*/*EDEM_GRID.tif")
egm_filesi = glob("/media/ljp238/12TBWolf/RSPROX/OUT_TILES/TILES12/*/*EGM08.tif")

tdemf_filesi =  glob("/media/ljp238/12TBWolf/RSPROX/OUT_TILES/TILES12/*/vfill/*DEM__Fw_mlinterps.tif")
tdemv_filesi = glob("/media/ljp238/12TBWolf/RSPROX/OUT_TILES/TILES12/*/*tdem_DEM__Fw.tif")

gdemv_filesi = glob("/media/ljp238/12TBWolf/RSPROX/GEDI_GRID/tiles/*/*GEDI03_elev_lowestmode_mean_2019108_2022019_002_03_EPSG4326.tif")
gdemf_filesi = glob('/media/ljp238/12TBWolf/RSPROX/GEDI_GRID/tiles/*/*GEDI03_elev_lowestmode_mean_2019108_2022019_002_03_EPSG4326_mlinterps.tif')
roi = 'TLS'
blocks_dpath = f"/media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/{roi}"
os.makedirs(blocks_dpath, exist_ok=True)

In [53]:
import subprocess

def list2txt(txt_file,filelist):
    with open(txt_file, 'w') as f:
        for tif in filelist:
            f.write(f"{tif}\n")
    print('list2txt')
    
def build_vrt(filelist,vrt_file):

    # g
    txt_file = vrt_file.replace('.vrt', '.txt')
   # gpkg_file = vrt_file.replace('.vrt', '.gpkg')

    if not os.path.isfile(vrt_file):
        list2txt(txt_file,filelist)
        subprocess.run(['gdalbuildvrt', '-input_file_list', txt_file, vrt_file])

    print(f"VRT created: {vrt_file}")
# do the other vrt python function too []

In [70]:
def build_tileindex(filelist, gkpg_file):
    txt_file = gkpg_file.replace('.gpkg', '.txt')
    if not os.path.isfile(gkpg_file):
        list2txt(txt_file,filelist)
        subprocess.run(['gdaltindex', gkpg_file,'--optfile', txt_file])
        print(f"GPKG created: {gkpg_file}")


In [71]:
gdemf_files = [i for i in gdemf_filesi for t in tilenames_tls if t in i]
gdemv_files = [i for i in gdemv_filesi for t in tilenames_tls if t in i]
tdemv_files = [i for i in tdemv_filesi for t in tilenames_tls if t in i]
tdemf_files = [i for i in tdemf_filesi for t in tilenames_tls if t in i]

assert len(tdemf_files) == len(tdemv_files) == len(gdemf_files) == len(gdemv_files),'files list != length'

grid_files = [i for i in grid_filesi for t in tilenames_tls if t in i]
egm_files = [i for i in egm_filesi for t in tilenames_tls if t in i]

assert len(tdemf_files) == len(tdemv_files) == len(grid_files) == len(egm_files),'files list != length'

In [72]:
vrt_grid = f'{blocks_dpath}/grid.vrt'
vrt_egm = f'{blocks_dpath}/egm08.vrt'
build_vrt(grid_files,vrt_grid)
build_vrt(egm_files,vrt_egm)

VRT created: /media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/grid.vrt
VRT created: /media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/egm08.vrt


In [73]:
vrt_tf = f'{blocks_dpath}/tdemf.vrt'
vrt_tv = f'{blocks_dpath}/tdemv.vrt'
vrt_gf = f'{blocks_dpath}/gdemf.vrt'
vrt_gv = f'{blocks_dpath}/gdemv.vrt'
build_vrt(gdemf_files,vrt_gf)
build_vrt(gdemv_files,vrt_gv)
build_vrt(tdemv_files,vrt_tv)
build_vrt(tdemf_files,vrt_tf)

VRT created: /media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/gdemf.vrt
VRT created: /media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/gdemv.vrt
VRT created: /media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/tdemv.vrt
VRT created: /media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/tdemf.vrt


In [74]:
gpkg = f'{blocks_dpath}/{roi}_tindex.gpkg'
build_tileindex(tdemf_files, gpkg)

list2txt
GPKG created: /media/ljp238/12TBWolf/RSPROX/OUT_TILES/BLOCKS/TLS/TLS_tindex.gpkg


##### translate to tifs 

In [63]:
# geoids 1 and 2, and then, make them all in tifs